In [ ]:
#Importar secuencias
qiime tools import 
--type 'SampleData[SequencesWithQuality]' 
--input-path Forward 
--input-format CasavaOneEightLanelessPerSampleDirFmt 
--output-path SecuenciasHongosForward

In [ ]:
# Hacer un resumen de lo que encontró de las secuencias
qiime demux summarize \
--i-data SecuenciasHongosCompletas.qza \
--o-visualization SecuenciasHongosCompletas.qvz

In [ ]:
qiime demux summarize \
--i-data SecuenciasHongosForward.qza \
--o-visualization SecuenciasHongosForward

In [ ]:
# DADA2
qiime dada2 denoise-single \
--i-demultiplexed-seqs SecuenciasHongosForward.qza \
--p-trim-left 6 \
--p-trunc-len 270 \
--o-table TablaDADA2HongosForward \
--o-representative-sequences SeqRepresentativasHongosForward \
--o-denoising-stats StatsDenoisingHongosForward \
--verbose

In [ ]:
qiime feature-table summarize \
--i-table TablaDADA2HongosForward.qza \
--o-visualization TablaDADA2HongosForward \
--m-sample-metadata-file metadatos_hongos_qiime_int2.txt

In [ ]:
qiime metadata tabulate \
--m-input-file StatsDenoisingHongosForward.qza \
--o-visualization StatsDenoisingHongosForward 

In [ ]:
qiime feature-table tabulate-seqs \
--i-data SeqRepresentativasHongosForward.qza \
--o-visualization SeqRepresentativasHongosForward.qzv

In [ ]:
#Arbol
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences DADA2/SeqRepresentativasHongosForward.qza \
  --o-alignment Alineadas.qza \
  --o-masked-alignment AlineadasMasked.qza \
  --o-tree ArbolUnrooted.qza \
  --o-rooted-tree ArbolRooted.qza --verbose

In [ ]:
# Diversidad 
qiime diversity core-metrics-phylogenetic \
--i-phylogeny ArbolRootedHongos.qza \
--i-table DADA2/TablaDADA2HongosForward.qza \
--p-sampling-depth 26919 \
    --m-metadata-file metadatos_hongos_qiime_int2.txt \
        --output-dir ResultadosDiversidadHongos

In [ ]:
# Clasificador UNITE 
cd Clasificador 

qiime tools import \
--type 'FeatureData[Sequence]' \
--input-path UNITE-ver9-2023/sh_refs_qiime_ver9_dynamic_25.07.2023.fasta \
--output-path SeqUNITE

qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-path  UNITE-ver9-2023/sh_taxonomy_qiime_ver9_dynamic_25.07.2023.txt \
--output-path TaxUNITE \
--input-format HeaderlessTSVTaxonomyFormat 

qiime feature-classifier fit-classifier-naive-bayes 
--i-reference-reads SeqUNITE.qza 
--i-reference-taxonomy TaxUNITE.qza 
--o-classifier ClasificadorUNITE 
--verbose

In [ ]:
# Clasificacion taxonómica
qiime feature-classifier classify-sklearn 
--i-classifier Clasificador/ClasViejo/unite_classifier.qza 
--i-reads DADA2/RepSeqHongos.qza 
--o-classification TaxonomiaHongos.qza

In [ ]:
#Taxonomia
qiime metadata tabulate \
--m-input-file TaxonomiaHongos.qza \
--o-visualization TaxonomiaHongos.qzv

# Barplots
qiime taxa barplot \
--i-table DADA2/TablaDADA2HongosForward.qza \
--i-taxonomy TaxonomiaHongos.qza \
--m-metadata-file metadatos_hongos_qiime_int2.txt \
--o-visualization BarplotHongos.qzv